<a href="https://colab.research.google.com/github/JuliethLopez/chatbot_simpsons/blob/master/Pre_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
# librerias
import re
import pickle
import numpy as np
import pandas as pd
import re, string
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

In [234]:
# frase del usuario
sentence = "Hello, how are you?"
sentence = re.sub(' +', ' ',sentence.lower()) #sentence en minusculas
sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence) #quita signos de puntuación
sentence

'hello how are you'

### Cargar modelo

In [50]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip
replace clasificador/variables/variables.data-00000-of-00002? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: clasificador/variables/variables.data-00000-of-00002  
  inflating: clasificador/variables/variables.index  
  inflating: clasificador/variables/variables.data-00001-of-00002  
  inflating: clasificador/saved_model.pb  


In [51]:
modelo_clasificador = tf.keras.models.load_model('clasificador')
#Check its architecture
modelo_clasificador.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8623)              74364752  
_________________________________________________________________
dropout (Dropout)            (None, 8623)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4311)              37178064  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4311)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2155)              9292360   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2155)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1077)              2

Cargamos tambien el encoder y el label_tokenizer para los labels

In [149]:
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/encoder.pickle', 'rb') as f:
  encoder = pickle.load(f)

In [150]:
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/label_tokenizer.pickle', 'rb') as f:
  label_tokenizer = pickle.load(f)

### Cargamos trigramas y bigramas

In [209]:
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqnew.pickle', 'rb') as f:
  freqnew = pickle.load(f)

In [210]:
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqbnew.pickle', 'rb') as f:
  freqbnew = pickle.load(f)

In [235]:
# preprocesamiento trigramas
vectorizer3 = CountVectorizer(vocabulary=freqnew.keys(), ngram_range=(3,3))
X3 = vectorizer3.fit_transform([sentence])
sequences = X3.toarray()

In [236]:
# preprocesamiento bigramas
vectorizer2 = CountVectorizer(vocabulary=freqbnew.keys(), ngram_range=(2,2))
X2 = vectorizer2.fit_transform([sentence])
sequences_b = X2.toarray()

In [237]:
print(sequences.shape)
print(sequences_b.shape)

(1, 8316)
(1, 307)


In [238]:
#concatenamos
sequences_bt = np.concatenate((sequences, sequences_b), axis=1)
sequences_bt.shape

(1, 8623)

In [239]:
sequences_bt

array([[0, 0, 0, ..., 0, 0, 0]])

In [240]:
if sum(sequences_bt[0]) != 0:
  pred = modelo_clasificador.predict_classes(sequences_bt) #prediccion del modelo
  pred = label_tokenizer.sequences_to_texts([pred]) #de token a palabra
  print("prediccion:", pred)
else:
  print("la suma es cero")

prediccion: ['<OOV>']
